In [1]:
# ----- (My project)
# Date: 09.01.25
# Plan: Make a Gradio UI, that lets you pick a job on seek.com, then scape key words and come up with a 
# plan on how to land jobs of the type selected.

# My project

In [ ]:
#pip install markdown

In [ ]:

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
import gradio as gr
import markdown

# ---- 1
# Initialize and constants & set up Gemini Flash LLM
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')
import os
import google.generativeai as genai
genai.configure(api_key= api_key)
# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",}
model = genai.GenerativeModel(model_name="gemini-1.5-flash",
  generation_config=generation_config,)
chat_session = model.start_chat(history=[  ])


# ---- 2
# A class to represent a Webpage
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"


# ---- 3
# Data + set up
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    return result

system_prompt = "You are an experience recrutiment and talent management assistant, who will be provided a list of roles on offer.\
You will display those roles along with a high level summary of the key steps you suggest to land those roles. \
Output is to be in markdown (i.e. a professional format, with bold headders, proper spacing between different sections, etc.)\
Include suggested next steps on how to successfully apply for and land each of these jobs."

def get_brochure_user_prompt(url):
    user_prompt = f"Here are the contents of your recruitment search. Please list out individual roles and your best advise on landing those roles."
    user_prompt += f"Please provide output in a professional style with bold text for headings, content nicely layed out under headings, different content split out into sections, etc.)\n"
    user_prompt += get_all_details(url)
    #user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    user_prompt = user_prompt[:7_500] # Truncate if more than 5,000 characters
    return user_prompt

def create_brochure(url):
    response = chat_session.send_message(system_prompt + get_brochure_user_prompt(url))
    result = response.text
    html_output = markdown.markdown(result)
    return html_output

# ---- 4 
# Gradio UI
with gr.Blocks(css="""
    #header-container { text-align: left; position: fixed; top: 10px; left: 0; padding: 10px; background-color: #f0f0f0; }
    #input-container { text-align: left; position: fixed; top: 100px; left: 0; right: 0; background: white; z-index: 100; padding: 8px; line-height: 0.5;}
    #output-container { margin-top: 160px; height: calc(100vh - 280px); overflow-y: auto; }
    #output-html { white-space: pre-wrap; font-family: monospace; border: 1px solid #ccc; padding: 5px; line-height: 1.2;}
    .button-container { margin-top: 10px; } /* Space above the button */
    .output-label { margin-top: 10px; font-weight: bold; } /* Style for output label */
""") as iface:
    with gr.Column(elem_id="main-container"):
        # Add header and description
        with gr.Row(elem_id="header-container"):
            gr.Markdown("# Job seeker guide")
            gr.Markdown("1.0 Works best with recruitment site https://www.seek.com.au/ (but can try others).")
            gr.Markdown("2.0 Search for jobs of your choice, copy URL from that search & paste in input field below to get helpful advice on how to land those roles.")


        
        with gr.Row(elem_id="input-container"):
            input_text = gr.Textbox(label="Input", elem_id="input-box")
        
        with gr.Column(elem_id="output-container"):
            output_label = gr.Markdown("<div class='output-label'>Output:</div>")
            output_text = gr.HTML(elem_id="output-html")
        
        # Move the button below the output box
        submit_btn = gr.Button("Generate", elem_id="generate-button", elem_classes="button-container")
    
    submit_btn.click(fn=create_brochure, inputs=input_text, outputs=output_text)

iface.launch(share=True)

